In [1]:
#!pip uninstall opencv_python_headless
#!pip3 install opencv-python
#!pip install tensorflow

# pip3 install --user opencv-python

In [2]:

import imgaug as ia
ia.seed(1)
# imgaug uses matplotlib backend for displaying images
%matplotlib inline
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa 
# imageio library will be used for image input/output
import imageio
import pandas as pd
import numpy as np
import requests
import re
import os
import glob
# this library is needed to read XML files for converting it into CSV
import xml.etree.ElementTree as ET
import shutil
from PIL import Image
from PIL import Image, ImageEnhance

from skimage import data, exposure

from PIL import Image
import numpy as np
from skimage import transform
from skimage.color import rgb2gray

from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np

from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import numpy as np
import mimetypes
import argparse
import imutils
import cv2
import os

# Importing all necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from PIL import Image
 
img_width, img_height = 256, 256

plt.rcParams['figure.figsize'] = [15,8]

import warnings
warnings.filterwarnings("ignore")

In [4]:
import cv2
import numpy as np
import os

import datetime
from skimage import io
import os
import random
import matplotlib.pyplot as plt
%matplotlib inline
import glob

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from tensorflow.keras.optimizers import Adam

In [5]:
model2 = load_model('classification_weight.hdf5')
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 127, 127, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 62, 62, 64)        0

### Image Pre-Processing 

In [6]:
# Prediction based on New Model

from skimage import data, exposure

from PIL import Image
import numpy as np
from skimage import transform
from skimage.color import rgb2gray

from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np

#model2 = load_model('classification_weight.hdf5')

def accept_bright(custom_img):
    
    # Bright Accept Images
    contrast = cv2.convertScaleAbs(custom_img, alpha=1, beta=100)
    blurred = cv2.GaussianBlur(contrast, (7, 7), 0)
    thresh = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=3)
    thresh = cv2.dilate(thresh, None, iterations=5)
    return thresh

def accept_low(custom_img):
    
    # Low Accept Images
    contrast = cv2.convertScaleAbs(custom_img, alpha=1, beta=110)
    blurred = cv2.GaussianBlur(contrast, (5, 5), 0)
    thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=5)
    thresh = cv2.dilate(thresh, None, iterations=5)
    return thresh

def reject_bright(custom_img):
    
    # Bright Reject Images
    contrast = cv2.convertScaleAbs(custom_img, alpha=1, beta=60)
    blurred = cv2.GaussianBlur(contrast, (7, 7), 0)
    thresh = cv2.threshold(blurred, 180, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=7)
    thresh = cv2.dilate(thresh, None, iterations=3)
    return thresh

def reject_low(custom_img):
    
    # Dark Reject Images
    #contrast = cv2.convertScaleAbs(custom_img, alpha=1, beta=100)
    #blurred = cv2.GaussianBlur(contrast, (3, 3), 0)
    #thresh = cv2.threshold(blurred, 150, 255, cv2.THRESH_BINARY)[1]
    #thresh = cv2.erode(thresh, None, iterations=7)
    #thresh = cv2.dilate(thresh, None, iterations=3)
    #return thresh

    # dark reject Images with Adaptive threshold
    contrast = cv2.convertScaleAbs(custom_img, alpha=1, beta=50)
    blurred = cv2.GaussianBlur(contrast, (3, 3), 0)
    thresh = cv2.threshold(blurred, 120, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=7)
    thresh = cv2.dilate(thresh, None, iterations=3)
    return thresh

def load(img):
    threshold = 140
    if np.mean(img) > threshold:
        #image = Image.open(img)
        image = Image.fromarray(img)
        images = image.convert(mode = 'L')
        image_np = np.array(images)
        sharp = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])       # sharpening the image
        custom_img = cv2.filter2D(image_np,-1,sharp)
        
        # Blob implementation
        detector = cv2.SimpleBlobDetector_create()
        # Detect blobs.
        keypoints = detector.detect(custom_img)
        num_blobs = len(keypoints)
        if num_blobs > 0:
            w = reject_bright(custom_img)
            np_image = Image.fromarray(w)
            np_image = np.array(np_image).astype('float32')/255
            np_image = transform.resize(np_image, (256, 256, 1))
            np_image = np.expand_dims(np_image, axis=0)                  # (1,256,256,1)
            return np_image
        else:
            x = accept_bright(custom_img)
            np_image = Image.fromarray(x)
            np_image = np.array(np_image).astype('float32')/255
            np_image = transform.resize(np_image, (256, 256, 1))
            np_image = np.expand_dims(np_image, axis=0)                  # (1,256,256,1)
            return np_image
        
    elif np.mean(img) < threshold:
        adapthist_image = exposure.equalize_hist(img)
        im = Image.fromarray((adapthist_image*255).astype(np.uint8))
        images = im.convert(mode = 'L')
        image_np = np.array(images)
        sharp = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])       # sharpening the image
        custom_img=cv2.filter2D(image_np,-1,sharp)
        # Blob implementation
        detector = cv2.SimpleBlobDetector_create()
        # Detect blobs.
        keypoints = detector.detect(custom_img)
        num_blobs = len(keypoints)
        if num_blobs > 0:
            y = reject_low(custom_img)
            np_image = Image.fromarray(y)
            np_image = np.array(np_image).astype('float32')/255
            np_image = transform.resize(np_image, (256, 256, 1))
            np_image = np.expand_dims(np_image, axis=0)                  # (1,256,256,1)
            return np_image
        else:
            img = Image.fromarray(img)
            images = img.convert(mode = 'L')
            image_np = np.array(images)
            sharp = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])       # sharpening the image
            custom_img=cv2.filter2D(image_np,-1,sharp)
            z = accept_low(custom_img)
            np_image = Image.fromarray(z)
            np_image = np.array(np_image).astype('float32')/255
            np_image = transform.resize(np_image, (256, 256, 1))
            np_image = np.expand_dims(np_image, axis=0)                  # (1,256,256,1)
            return np_image


#inp = plt.imread('C:\\Users\\nabhishe\\MITC_Project\\RGB_Test\\Reject-A26491-001-Sample - 103-1.jpg')
#image2 = load(inp)
#label = model2.predict(image2)
#plt.imshow(image2.reshape(256,256),cmap='gray')
#plt.show()
#print("Predicted Class (0 - Accept , 1 - Reject): ", label[0][0])

### Segementing a Region based on Region of Interest (ROI)

In [7]:
# to segment a region of interest in the image

def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

### Running Average

In [8]:
def run_avg(image, accumWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, accumWeight)

In [9]:
# Try 2

### Real Time Implementation

In [9]:
###########################################################################################################
###########################################################################################################

labels_dict={0:'ACCEPT',1:'REJECT'}
color_dict={0:(255,0,0),1:(0,0,255)}

# # load Model Weights
from PIL import Image as im

# global variables
bg = None

#def _load_weights():
#    try:
#        model2 = load_model("classification_weight.hdf5")
#        print(model2.summary())
#        print(model2.get_weights())
#        #print(model.optimizer)
#        return model2
#    except Exception as e:
#        return None
    
def _load_weights():
    model2 = load_model("classification_weight.hdf5")
    #print(model2.summary())
    #print(model2.get_weights())
    #print(model.optimizer)
    return model2
        
#def getPredictedClass(model2):
    
    #image = plt.imread('C:\\Users\\nabhishe\\MITC_Project\\RGB_Test\\Accept-A26491-001-Sample-9-1.jpg',1)
    
    #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #gray_image = cv2.resize(gray_image, (100, 120))
    #gray_image = gray_image.reshape(1, 100, 120, 1)
    
    #res_img = load(image)
    #label = model2.predict(res_img)
    #print("Predicted Class (0 - Accept , 1 - Reject): ", label[0][0])
    
    #print(type(label))
    #predicted_class = np.argmax(label[0][0])
    #if predicted_class < 0.2:
    #    return "ACCEPT"
    #elif predicted_class > 0.2:
    #    return "REJECT"
    
    
    #res_im=cv2.bitwise_not(res_img)
    #img_np = np.array(res_img).reshape(256,256)
    #img_np = np.array(res_img)
    #img_np = (img_np/256).astype('uint8')
    #detector = cv2.SimpleBlobDetector_create()
    # Detect blobs.
    #keypoints = detector.detect(img_np)
    #num_blobs = len(keypoints)
    #if num_blobs > 0:
    #    return 'REJECT'
    #elif num_blobs < 0:
    #    return 'ACCEPT'
    

    #prediction = model2.predict_on_batch(res_img)
    #predicted_class = np.argmax(prediction)
    #if predicted_class == 0:
    #    return "ACCEPT"
    #elif predicted_class == 1:
    #    return "REJECT"
    
    #if label[0][0] < 0.2:
    #    return 0
    #elif label[0][0] > 0.2:
    #    return 1
        
        
if __name__ == "__main__":
    # Video Detection

    # initialize accumulated weight
    accumWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(1)

    #codec = 0x47504A4D  # MJPG
    #x = camera.set(cv2.CAP_PROP_FPS, 30.0)
    #x = camera.set(cv2.CAP_PROP_FOURCC, codec)

    fps = int(camera.get(cv2.CAP_PROP_FPS))

    # region of interest (ROI) coordinates
    top, right, bottom, left = 100, 150, 400, 490

    # initialize num of frames
    num_frames = 0

    # calibration indicator
    calibrated = False
    model = _load_weights()
    k = 0

    # keep looping, until interrupted
    while (True):
        # get the current frame
        (grabbed, frame) = camera.read()

        scale=10
        
        #get the webcam size
        height, width, channels = frame.shape

        #prepare the crop
        centerX,centerY=int(height/2),int(width/2)
        radiusX,radiusY= int(scale*height/100),int(scale*width/100)

        minX,maxX=centerX-radiusX,centerX+radiusX
        minY,maxY=centerY-radiusY,centerY+radiusY

        cropped = frame[minX:maxX, minY:maxY]
        resized_cropped = cv2.resize(cropped, (width, height), fx = 1, fy = 1, interpolation = cv2.INTER_AREA) 
        
        # flip the frame so that it is not the mirror view
        #resized_cropped = cv2.flip(resized_cropped, 1)
        
        # clone the frame
        clone = resized_cropped.copy()

        # get the ROI
        roi = resized_cropped[top:bottom, right:left]
        
        # Capture Denoising
        #roi = cv2.fastNlMeansDenoisingColored(roi,None,10,10,7,21)

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our weighted average model gets calibrated
        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successfull...")
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                #cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                # count the number of fingers
                # fingers = count(thresholded, segmented)
                if k % (fps / 6) == 0:
                    cv2.imwrite('C:\\Users\\nabhishe\\MITC_Project\\test_img.jpg', thresholded)
                    #predictedClass = getPredictedClass(model)
                    #image = plt.imread('C:\\Users\\nabhishe\\MITC_Project\\RGB_Test\\Accept-A26491-001-Sample-9-1.jpg')
                    #image = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                    res_img = load(roi)
                    #model = _load_weights()
                    label = model.predict(res_img)
                    score = label[0][0]
                    if label[0][0] < 0.2:
                        pred = 0
                    elif label[0][0] > 0.2:
                        pred = 1
                    print(pred)   
                    #score = label[0][0]
                    #score_txt = f'{100 * round(score)}%'

                    #img_boxes = cv2.rectangle(clone,(left,top),(right,bottom),(0,255,0),1)

                    # rgb - clone
                    # xmin - left
                    # ymax - top
                    # xmax - right
                    # ymin - bottom

                    #cv2.putText(clone, str(predictedClass), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    if pred == 0:
                        #score_txt = f'{100 * round(score)}%'
                        img_boxes = cv2.rectangle(clone,(left,top),(right,bottom),(0,255,0),2)          # Green Box
                        cv2.putText(img_boxes, labels_dict[pred], (left,top-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA) #Put a label over box.
                        #cv2.putText(img_boxes, score_txt, (left,top+10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    elif pred == 1:
                        #score_txt = f'{100 * round(score)}%'
                        img_boxes = cv2.rectangle(clone,(left,top),(right,bottom),(0,0,255),2)          # Red Box
                        cv2.putText(img_boxes, labels_dict[pred], (left,top-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        #cv2.putText(img_boxes, score_txt, (left,top+10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                # show the thresholded image
                cv2.imshow("Thesholded", thresholded)

        k = k + 1
        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (255, 0, 0), 2)          # cv2.rectangle(rgb,(xmin,ymax),(xmax,ymin),(0,255,0),1)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(10)

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

# free up memory
camera.release()
cv2.destroyAllWindows()

[STATUS] please wait! calibrating...
[STATUS] calibration successfull...
0
1
0
1
1
1
1
1
0
0
0
0
0
1
1
1
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1


In [9]:
import cv2

In [15]:
# Video

In [7]:
from IPython.display import Video

Video('C:\\Users\\nabhishe\\Videos\\Captures\\Test 1.mp4', width=900, height=900, embed=True)